pip install tensorflow

In [27]:
import os
import sys
sys.path.append("../../")
# openCV
import cv2
# numpy
import numpy as np
# class util functions
#from utils.imutils import jimshow as show
#from utils.imutils import jimshow_channel as show_channel
#import utils.classifier_utils as clf_util

# Import sklearn metrics
from sklearn import metrics
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# cifar10 dataset
from tensorflow.keras.datasets import cifar10

ModuleNotFoundError: No module named 'utils'

In [22]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [28]:
# define labels
classes = [dog, frog, cat]
classes

NameError: name 'dog' is not defined

In [24]:
X_list=[]

for image in X_train:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
    normalized = gray_image/255 # normalize
    X_list.append(normalized) # add to list
X_preprocess_array = np.array(X_list) # turn list back into an array
# reshape
X_preprocess_train = X_preprocess_array.reshape(-1, 1024)

type(image)

numpy.ndarray

In [25]:
# making a logistic regression classifier
clf = LogisticRegression(tol=0.1, 
                         solver='saga',
                         multi_class='multinomial').fit(X_preprocess_train, y_train)

/home/ucloud/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
# try and predict the class of a data point
clf_util.plot_individual(X_preprocess_train, 
                         y_test, 
                         50)

NameError: name 'clf_util' is not defined